In [12]:
import pandas as pd
import json
import pickle
import nbimporter

In [13]:
from recipe_class import Recipe
from generate_menu import generate_weekly_menu

In [14]:
def create_ingredient_list():
    
    ingredients_list = {}
    recipe_servings_count = {}
    
    weekly_menu = generate_weekly_menu()

    # Iterate through each day and meal in the weekly menu
    for day in weekly_menu.columns:
        for meal_type in weekly_menu.index:
            recipe_obj = weekly_menu.at[meal_type, day]

            # Check if the recipe has servings left from a previous day
            if recipe_servings_count.get(recipe_obj.name, 0) > 0:
                
                # Decrease the count of remaining servings and continue to next recipe
                recipe_servings_count[recipe_obj.name] -= 1
                continue

            # Iterate over the ingredients of the recipe and aggregate their quantities
            for ingredient, details in recipe_obj.ingredients.items():
                qty, unit = details['qty'], details['unit']
                if ingredient in ingredients_list:
                    ingredients_list[ingredient]['qty'] += qty
                else:
                    ingredients_list[ingredient] = {'qty': qty, 'unit': unit}

            # Set the number of servings for this recipe (minus the current one)
            recipe_servings_count[recipe_obj.name] = recipe_obj.servings - 1
            
    # Create ingredients DF
    ingredients_df = pd.DataFrame.from_dict(ingredients_list, orient='index')
    ingredients_df = ingredients_df.reset_index()
    ingredients_df.rename(columns={'index': 'ingredients'}, inplace=True)

    return ingredients_df, weekly_menu